In [1]:
pip install pillow

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [112]:
from PIL import Image
import numpy as np

image_path = ('collage 10240.png')
image = Image.open(image_path)
test_array = np.random.rand(1000,1000)

image_np = np.array(image)

print(image_np.shape)

(7680, 10240, 3)


In [113]:
kernel_relief = np.array([[-2, -1, 0],
                          [-1,  1, 1],
                          [ 0,  1, 2]])



# Padding

In [114]:
import numpy as np

test = np.random.randint(1, 10, size = (4, 10))
arr2 = np.zeros((test.shape[0]+2, test.shape[1]+2), dtype=int)

for i in range(test.shape[0]):
    for j in range(test.shape[1]):
        arr2[i+1][j+1] = test[i,j]

print(test)
print('\n')
print(arr2)
print('\n')
print(f'test shape' + str(test.shape))
print(f'arr shape' + str(arr2.shape))

[[5 8 8 3 9 1 1 5 3 7]
 [6 2 2 2 2 8 4 5 2 1]
 [1 8 1 2 6 5 6 5 3 2]
 [8 1 5 5 9 5 1 5 7 7]]


[[0 0 0 0 0 0 0 0 0 0 0 0]
 [0 5 8 8 3 9 1 1 5 3 7 0]
 [0 6 2 2 2 2 8 4 5 2 1 0]
 [0 1 8 1 2 6 5 6 5 3 2 0]
 [0 8 1 5 5 9 5 1 5 7 7 0]
 [0 0 0 0 0 0 0 0 0 0 0 0]]


test shape(4, 10)
arr shape(6, 12)


# Свёртка

In [115]:
kernel_relief = np.array([[-2, -1, 0],
                          [-1,  1, 1],
                          [ 0,  1, 2]])
kernel_size = kernel_relief.shape[1]
test = np.random.randint(1, 10, size = (5, 10))

rows_num = test.shape[0]
column_num = test.shape[1]

padding = 0
stride = 1
output_height=((rows_num-kernel_size+2*padding)/stride)+1
output_width=((column_num-kernel_size+2*padding)/stride)+1

arr = np.zeros((int(output_height),int(output_width)), dtype=int)


print(test)
print('\n')
print(arr)
print('\n')
test.shape

[[4 1 6 2 4 3 2 9 9 2]
 [7 5 5 6 7 4 6 7 8 6]
 [9 3 1 6 8 9 2 1 9 8]
 [2 2 5 6 3 9 4 4 5 3]
 [4 7 7 5 2 8 8 7 2 3]]


[[0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]]




(5, 10)

# Свёртка

In [103]:
test3 = np.random.randint(1,10, size = (3,3))
a = 0
for i in range(kernel_relief.shape[1]):
    for j in range(kernel_relief.shape[1]):
        a += test3[i][j]*kernel_relief[i][j]

print(test3)
print('\n')
print(kernel_relief)
print('\n')
a

[[3 1 6]
 [9 5 8]
 [4 3 1]]


[[-2 -1  0]
 [-1  1  1]
 [ 0  1  2]]




2

In [120]:

frame = np.zeros_like(kernel_relief)

i = j = k = l = val = 0
print(test)
print('\n')
print(arr)
print('\n')

while k < arr.shape[0]:
    while l < arr.shape[1]:
        while i < kernel_size:
            while j < kernel_size:
                frame[i][j] = test[i+k][j+l] # здесь заполнеяем один кадр изображения
                j += 1

            j = 0
            i += 1
        
        if (i == kernel_size): # как только кадр заполнился
            for m in range(frame.shape[0]):
                for n in range(frame.shape[1]):
                    val += frame[m][n]*kernel_relief[m][n]  # считаем значение свёртки
        arr[k][l] = val
        val = 0
        j = 0
        i = 0
        l += 1         

    l = 0
    k += 1


print(arr)
print('\n')

[[4 1 6 2 4 3 2 9 9 2]
 [7 5 5 6 7 4 6 7 8 6]
 [9 3 1 6 8 9 2 1 9 8]
 [2 2 5 6 3 9 4 4 5 3]
 [4 7 7 5 2 8 8 7 2 3]]


[[ -1  11  16  23   5   5  15   5]
 [-12   6   9  13   2  -8   3   5]
 [  5  19   5   4   9   1  11   1]]


[[ -1  11  16  23   5   5  15   5]
 [-12   6   9  13   2  -8   3   5]
 [  5  19   5   4   9   1  11   1]]


